In [41]:
from yahoodownloader import YahooDownloader
from preprocessors import FeatureEngineer, data_split
from models import DRLEnsembleAgent
import numpy as np

In [42]:
TRAIN_START_DATE = '2010-01-01' # <3
TRAIN_END_DATE = '2021-01-01'
TEST_START_DATE = '2021-01-01'
TEST_END_DATE = '2023-03-01'

In [43]:
nifty_50_tickers = [
    'ADANIPORTS.NS',
    'ASIANPAINT.NS',
    'AXISBANK.NS',
    'BAJAJ-AUTO.NS',
    'BAJFINANCE.NS',
    'BAJAJFINSV.NS',
    'BPCL.NS',
    'BRITANNIA.NS',
    'CIPLA.NS',
    'COALINDIA.NS',
    'DIVISLAB.NS',
    'DRREDDY.NS',
    'EICHERMOT.NS',
    'GRASIM.NS',
    'HCLTECH.NS',
    'HDFCBANK.NS',
    'HEROMOTOCO.NS',
    'HINDALCO.NS',
    'HINDUNILVR.NS',
    'ICICIBANK.NS',
    'IOC.NS',
    'INDUSINDBK.NS',
    'INFY.NS',
    'ITC.NS',
    'JSWSTEEL.NS',
    'KOTAKBANK.NS',
    'LT.NS',
    'M&M.NS',
    'MARUTI.NS',
    'NESTLEIND.NS',
    'NTPC.NS',
    'ONGC.NS',
    'POWERGRID.NS',
    'RELIANCE.NS',
    'SHREECEM.NS',
    'SBIN.NS',
    'SBILIFE.NS',
    'SUNPHARMA.NS',
    'TCS.NS',
    'TATACONSUM.NS',
    'TATAMOTORS.NS',
    'TATASTEEL.NS',
    'TECHM.NS',
    'TITAN.NS',
    'ULTRACEMCO.NS',
    'UPL.NS',
    'WIPRO.NS'
]

DOW_30_TICKER = [
    "AXP",
    "AMGN",
    "AAPL",
    "BA",
    "CAT",
    "CSCO",
    "CVX",
    "GS",
    "HD",
    "HON",
    "IBM",
    "INTC",
    "JNJ",
    "KO",
    "JPM",
    "MCD",
    "MMM",
    "MRK",
    "MSFT",
    "NKE",
    "PG",
    "TRV",
    "UNH",
    "CRM",
    "VZ",
    "V",
    "WBA",
    "WMT",
    "DIS",
    "DOW",
]

In [44]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = nifty_50_tickers).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [45]:
df.head()
# df = df.sort_values(["date", "tic"], ignore_index=True)
# df.index = df.date.factorize()[0]
# merged_closes = df.pivot_table(index="date", columns="tic", values="close")
# merged_closes = merged_closes.interpolate(method='linear', axis=1)
# merged_closes = merged_closes.dropna(axis=1)
# tics = merged_closes.columns
# df = df[df.tic.isin(tics)]
# df.head()

,date,open,high,low,close,volume,tic,day
0,2010-01-04,111.000000,112.559998,110.699997,102.799355,1079040,ADANIPORTS.NS,0
1,2010-01-04,179.100006,179.990005,178.000000,159.429932,80350,ASIANPAINT.NS,0
2,2010-01-04,198.779999,199.990005,197.619995,180.921814,4371510,AXISBANK.NS,0
3,2010-01-04,882.500000,886.474976,862.525024,579.148254,381510,BAJAJ-AUTO.NS,0
4,2010-01-04,34.332169,35.491478,34.017780,34.740265,2469924,BAJAJFINSV.NS,0


In [46]:
INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

In [47]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

Successfully added technical indicators


/Users/omgodage/Desktop/College/sem 5/CS337/piroject/preprocessors.py:218: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_price_pivot = df_price_pivot.pct_change()


Successfully added turbulence index


In [48]:
processed.sample(5)

,date,open,high,low,close,volume,tic,day,macd,rsi_30,cci_30,dx_30,turbulence
15597,2011-12-20,457.022217,458.977783,431.333344,364.048492,12356095,LT.NS,1,-26.702938,28.474785,-240.166895,48.160530,78.979494
57075,2017-03-28,109.490654,110.812531,109.490654,70.374199,6020578,POWERGRID.NS,1,-0.219496,52.987446,-6.026417,10.380247,19.117182
20819,2012-08-13,66.937515,67.640640,66.431267,37.781052,2775124,POWERGRID.NS,0,0.727647,61.796030,101.167905,27.576890,16.666574
38518,2014-11-19,9199.950195,9279.900391,8930.599609,8768.500977,8901,SHREECEM.NS,2,192.963881,61.151968,79.314973,9.260996,19.660898
3883,2010-06-30,104.000000,106.500000,103.330002,85.419044,9869710,JSWSTEEL.NS,2,-0.757447,47.354251,-12.600642,8.311192,0.000000


In [49]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
# processed.timkz

Stock Dimension: 33, State Space: 199


In [50]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

In [51]:
rebalance_window = 63 # rebalance_window is the number of days to retrain the model
validation_window = 63 # validation_window is the number of days to do validation and trading (e.g. if validation_window=63, then both validation and trading period will be 63 days)

ensemble_agent = DRLEnsembleAgent(df=processed,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TEST_START_DATE,TEST_END_DATE),
                 rebalance_window=rebalance_window, 
                 validation_window=validation_window, 
                 **env_kwargs)


In [52]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

timesteps_dict = {'a2c' : 10_000, 
                 'ppo' : 10_000, 
                 'ddpg' : 10_000
                 }

In [53]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 timesteps_dict)

============Start Ensemble Strategy============
turbulence_threshold:  179.0966017533276
======Model training from:  2010-01-01 to  2021-01-04
======A2C Training========
{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0007}
Using cpu device


ValueError: could not broadcast input array from shape (194,) into shape (199,)